In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?"

In [3]:
data_frame_city = pd.read_csv("data_frame_city.csv")
data_frame_city

,Unnamed: 0,city,lat,lng,max temp,humidity,cloudiness,wind speed,country,Date
0,0,Ürümqi,43.8010,87.6005,265.68,79,100,0.00,CN,1638641613
1,1,Wuhu,31.3366,118.3755,273.10,57,7,2.63,CN,1638641803
2,2,Port Elizabeth,-33.9180,25.5701,291.31,77,90,6.17,ZA,1638641804
3,3,Olafsvik,64.8945,-23.7142,274.82,78,75,2.76,IS,1638641804
4,4,Avarua,-21.2078,-159.7750,298.18,88,75,1.03,CK,1638641804
...,...,...,...,...,...,...,...,...,...,...
588,588,Vavoua,7.3819,-6.4778,302.86,42,57,1.47,CI,1638641857
589,589,Dudinka,69.4058,86.1778,237.38,100,93,1.36,RU,1638641740
590,590,Nānākuli,21.3906,-158.1547,292.99,80,90,2.57,US,1638641857
591,591,Mkushi,-13.6202,29.3939,296.92,32,12,2.69,ZM,1638641857


In [4]:
gmaps.configure(g_key)
fig = gmaps.figure()
subset = data_frame_city[["lat","lng"]]

tuples = [tuple(x) for x in subset.values]
locations = tuples
fig.add_layer(gmaps.heatmap_layer(locations, weights = data_frame_city["humidity"]))
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
vacation_df = data_frame_city[data_frame_city["max temp"] >250]
vacation_df

,Unnamed: 0,city,lat,lng,max temp,humidity,cloudiness,wind speed,country,Date
0,0,Ürümqi,43.8010,87.6005,265.68,79,100,0.00,CN,1638641613
1,1,Wuhu,31.3366,118.3755,273.10,57,7,2.63,CN,1638641803
2,2,Port Elizabeth,-33.9180,25.5701,291.31,77,90,6.17,ZA,1638641804
3,3,Olafsvik,64.8945,-23.7142,274.82,78,75,2.76,IS,1638641804
4,4,Avarua,-21.2078,-159.7750,298.18,88,75,1.03,CK,1638641804
...,...,...,...,...,...,...,...,...,...,...
586,586,Otradnoye,59.7775,30.8181,265.18,94,100,2.76,RU,1638641857
588,588,Vavoua,7.3819,-6.4778,302.86,42,57,1.47,CI,1638641857
590,590,Nānākuli,21.3906,-158.1547,292.99,80,90,2.57,US,1638641857
591,591,Mkushi,-13.6202,29.3939,296.92,32,12,2.69,ZM,1638641857


In [6]:
#If you can see the temperature is ranging from 200-300 F so my filter value is a lot higher
vacation_df = data_frame_city.loc[(data_frame_city["max temp"] < 500)  \
                                  & (data_frame_city["cloudiness"] == 0) \
                                  & (data_frame_city["max temp"] >250)]


In [7]:
hotel_df =[]

for index, row in vacation_df.iterrows():    

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="+str(row["lat"])+"%2C"+str(row["lng"])+"&radius=5000&type=lodging&key="+g_key

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    try: 
        hotel_df.append(response.json()["results"][0]["name"])
    except:
        hotel_df.append("")
        pass

In [19]:
vacation_df["hotel name"] = hotel_df

vacation_df

C:\Users\905294\AppData\Local\Temp/ipykernel_27916/3643344187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_df["hotel name"] = hotel_df


,Unnamed: 0,city,lat,lng,max temp,humidity,cloudiness,wind speed,country,Date,hotel name
11,11,Hasaki,35.7333,140.8333,277.37,51,0,9.39,JP,1638641804,Hotel Sunrise Choshi
26,26,Lebu,-37.6167,-73.6500,288.84,64,0,9.21,CL,1638641810,Hostal Las Lilas
49,49,Busselton,-33.6500,115.3333,288.20,77,0,4.78,AU,1638641813,Observatory Guest House
54,54,Carnarvon,-24.8667,113.6333,295.19,83,0,3.60,AU,1638641813,Hospitality Carnarvon
56,56,Salalah,17.0151,54.0924,298.20,61,0,2.06,OM,1638641813,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
65,65,Wad Rāwah,15.1603,33.1397,303.37,23,0,3.10,SD,1638641814,ود الترابي
69,69,Henties Bay,-22.1160,14.2845,290.85,84,0,5.63,NaN,1638641814,Fishermans Guest House
122,122,Aswān,24.0934,32.9070,296.46,28,0,1.79,EG,1638641818,Nuba Nile
130,130,Yinzhu,35.8786,119.9753,276.16,61,0,3.23,CN,1638641819,Vienna Hotel Qingdao Huangdao
138,138,Coquimbo,-29.9533,-71.3436,292.00,68,0,6.69,CL,1638641819,Hostel Del Puerto


In [17]:
hotel_df

['Hotel Sunrise Choshi',
 'Hostal Las Lilas',
 'Observatory Guest House',
 'Hospitality Carnarvon',
 'HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel',
 'ود الترابي',
 'Fishermans Guest House',
 'Nuba Nile',
 'Vienna Hotel Qingdao Huangdao',
 'Hostel Del Puerto',
 'Ilmajoen Kestikartano',
 'BUNGALOW MAHAMANINA',
 'Gran Hotel Mar del Plata',
 'Maltahöhe Hotel',
 'Katsuura Hotel Mikazuki',
 'Sur Plaza Hotel',
 'Broadwater Mariner Resort',
 'محل الشريف محمد العايب',
 'Hotel Khovd',
 'Crowne Plaza Ordos',
 'International Lodge Motel',
 'محمد هادي ال فطيح',
 'Hotel Arena Gruesa',
 'Hótel Höfn',
 'Grace Inn Zhangqiu Station Avenue Branch',
 'Hotel Kokkola',
 'Hosteria Sampedro',
 'Le Meridien Qingdao',
 'B&B La Casona de Nazaret',
 'Hotel JM Puerto del Rosario',
 'Diego De Almagro Coyhaique',
 '',
 'Hotel Narayan Avezika Comfort',
 'Leisure Vacation Myrica Hotel',
 'Birjand Tourist Hotel',
 'Lazy River Motor Inn',
 'Spahotel Casino',
 'Hôtel Abda',
 'Oyster Box Guesthouse',
 'Super Leisure Hotel',


In [26]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["lat", "lng"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))